In [1]:
from emojis_lib import *

# Construct Model

In [2]:
lemmatizer = init_lemmatizer(150000)

In [3]:
df = mongo_to_df("emojis")

In [4]:
tweets = df_to_clean_list(df)

In [5]:
len(tweets)

3036909

In [6]:
tweets = [clean_tweet(tweet,lemmatizer) for tweet in tweets]

In [7]:
tweets = [emoji.emojize(emoji.demojize(tweet).replace("::", ": :")) for tweet in tweets]

In [59]:
model = apply_w2v(tweets, 100, 4, 4, 12, 1e-3)

Training model...
Done!


In [60]:
model.most_similar(positive=['🇺🇸'], topn=100)

[('❤️🇺🇸', 0.736808180809021),
 ('godblessamerica', 0.7155251502990723),
 ('supertuesday16', 0.7048896551132202),
 ('merica', 0.6967030167579651),
 ('trump2016', 0.6897627115249634),
 ('makeamericagreatagain', 0.6892233490943909),
 ('marcomentum', 0.6818943023681641),
 ('secprimary', 0.6811294555664062),
 ('choosecruz', 0.6767135262489319),
 ('votetrump', 0.6758991479873657),
 ('🗳', 0.6756072044372559),
 ('election2016', 0.674425482749939),
 ('alwaystrump', 0.6732003688812256),
 ('maga', 0.666287899017334),
 ('texasprimary', 0.6648662090301514),
 ('votetrumpsupertuesday', 0.6550084352493286),
 ('rubio2016', 0.6524947881698608),
 ('ivoted', 0.6516146659851074),
 ('bospoli', 0.6455981731414795),
 ('tedcruz2016', 0.637853741645813),
 ('rockthevote', 0.6332900524139404),
 ('supertuesday', 0.6332550048828125),
 ('votetrump2016', 0.6298918724060059),
 ('cruzcrew', 0.6286619901657104),
 ('okprimary', 0.6246793866157532),
 ('stoptrump', 0.6215386986732483),
 ('gaprimary', 0.6166781187057495),
 

# Emoji stats

In [61]:
faces = ["😀", "😬", "😁", "😂", "😃", "😄", "😅", "😆", "😇", "😉", "😊", "🙂", "🙃", "☺️", "😋", "😌", "😍", "😘",
        "😗", "😙", "😚", "😜", "😝", "😛", "🤑", "🤓", "😎", "🤗", "😏", "😶", "😐", "😑", "😒", "🙄", "🤔", "😳",
        "😞", "😟", "😠", "😡", "😔", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "😤", "😮", "😱", "😨", "😰", "😯",
        "😦", "😧", "😢", "😥", "😪", "😓", "😭", "😵", "😲", "🤐", "😷", "🤒", "🤕", "😴"]

In [62]:
count = count_emojis(tweets, 850)["text"].value_counts().iloc[:850].keys()

In [63]:
list_emojis = count[:650]

In [64]:
def most_similar(positive, negative=[], nb_words=50000, top=5):
    a,_ = map(list, zip(*model.most_similar(positive=positive, negative=negative, topn=nb_words)))
    return [word for word in a if word in list_emojis][:top]

In [65]:
most_similar(positive = ["😂"], top = 15)

['💀', '😅', '🤔', '🙄', '😭', '😩', '😳', '😒', '🚮', '🍆', '😐', '🤐', '😴', '😷', '😹']

In [66]:
def most_similar_face(positive, negative=[], nb_words=100000, top=5):
    a,_ = map(list, zip(*model.most_similar(positive=positive, negative=negative, topn=nb_words)))
    return [word for word in a if word in faces][:top]

In [67]:
most_similar_face(positive = ["bore"], top = 10)

['😴', '😠', '😡', '😒', '😧', '😑', '🙄', '😤', '😂', '🤔']

# Emoji map

In [68]:
import matplotlib.pyplot as plt

In [69]:
emo_vecs = []
for emo in list_emojis:
    try:
        emo_vecs.append(model[emo])
    except Exception:
        continue

In [79]:
ts = TSNE(2, perplexity=50, early_exaggeration=1, n_iter=7000,
          init = 'pca',metric='euclidean')
reduced_vecs = ts.fit_transform(emo_vecs)

In [80]:
fig = plt.figure(figsize = (25,10))
ax = fig.add_subplot(111)

plt.plot(reduced_vecs[:,0], reduced_vecs[:,1], "o", markersize=0)
for i, txt in enumerate(list_emojis):
    try:
        ax.annotate(txt, (reduced_vecs[:,0][i],reduced_vecs[:,1][i]), fontsize = 10)
    except Exception:
        continue
    
plt.show()

In [46]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=12, init='k-means++', max_iter=200, n_init=1)

In [22]:
km.fit(emo_vecs)

KMeans(copy_x=True, init='k-means++', max_iter=200, n_clusters=12, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)